In [76]:
import os
import matplotlib.pyplot as plt
import keras.applications.vgg16
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.layers import Input, Flatten, Dense, Dropout, Activation
from keras.models import Sequential, Model
from keras.optimizers import Adam, SGD
from keras.applications.vgg16 import preprocess_input, decode_predictions
from keras.preprocessing.image import ImageDataGenerator
import numpy as np

os.chdir('D:\\machine learning')

classes = ['Tulip', 'Snowdrop', 'LilyValley', 'Bluebell', 'Crocus',
           'Iris', 'Tigerlily', 'Daffodil', 'Fritillary', 'Sunflower',
           'Daisy', 'ColtsFoot', 'Dandelion', 'Cowslip', 'Buttercup',
           'Windflower', 'Pansy']

train_batch_size = 128
validate_batch_size = 32

nb_classes = len(classes)


img_rows, img_cols, img_cnls = 224, 224, 3

FROM_DIR = '.\\jpg'
TRAIN_DIR = '.\\train_image'
VAL_DIR = '.\\test_image'
RESULT_DIR = '.\\results'
NEW_DIR = '.\\new images'

if not os.path.exists(TRAIN_DIR):
    os.mkdir(TRAIN_DIR)

if not os.path.exists(VAL_DIR):
    os.mkdir(VAL_DIR)
    
if not os.path.exists(RESULT_DIR):
    os.mkdir(RESULT_DIR)

nb_train_samples = 1190
nb_val_samples = 170
nb_epoch = 1000

def percent(value):
    return '%.2f%%' % (value * 100)

input_tensor = Input(shape = (img_rows, img_cols, img_cnls))
vgg_model = VGG16(weights = 'imagenet', include_top = False, input_tensor = input_tensor)
print(vgg_model.output_shape[1:])

(7, 7, 512)


In [77]:
#building fully-connected layers
flatten = Flatten()(vgg_model.output)
dense_1 = Dense(256)(flatten)
activation_1 = Activation('relu')(dense_1)
dropout_1 = Dropout(0.5)(activation_1)
dense_2 = Dense(nb_classes)(dropout_1)
activation_2 = Activation('softmax')(dense_2)

combined_model = Model(inputs = vgg_model.input, outputs = activation_2)
combined_model.summary()

Model: "model_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_12 (InputLayer)        (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0  

In [78]:
for layer in combined_model.layers[:15]:
    layer.trainable = False
learning_rate = 0.0001
combined_model.compile(loss = 'categorical_crossentropy',
                      optimizer = SGD(learning_rate, momentum = 0.9),
                      metrics = ['accuracy'])

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.5,
                                   zoom_range = 0.7,
                                   rotation_range = 45,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1.0/255)

train_generator = train_datagen.flow_from_directory(TRAIN_DIR,
                                                   target_size = (img_rows, img_cols),
                                                   color_mode = 'rgb',
                                                   classes = classes,
                                                   class_mode = 'categorical',
                                                   batch_size = train_batch_size,
                                                   shuffle = True)

validation_generator = test_datagen.flow_from_directory(VAL_DIR,
                                                       target_size = (img_rows,img_cols),
                                                       color_mode = 'rgb',
                                                       classes = classes,
                                                       class_mode = 'categorical',
                                                       batch_size = validate_batch_size,
                                                       shuffle = True)

Found 1190 images belonging to 17 classes.
Found 170 images belonging to 17 classes.


In [ ]:
len(train_generator), len(validation_generator)

(10, 6)

In [ ]:
hitsory = combined_model.fit_generator(train_generator,
                                       epochs = nb_epoch,
                                       verbose = 1,
                                       validation_data = validation_generator)

Epoch 1/1000
10/10 [==============================] - 13s 1s/step - loss: 3.2962 - accuracy: 0.0546 - val_loss: 2.9871 - val_accuracy: 0.0647
Epoch 2/1000
10/10 [==============================] - 12s 1s/step - loss: 2.9898 - accuracy: 0.0613 - val_loss: 2.9099 - val_accuracy: 0.0765
Epoch 3/1000
10/10 [==============================] - 12s 1s/step - loss: 2.8760 - accuracy: 0.0605 - val_loss: 2.7927 - val_accuracy: 0.0824
Epoch 4/1000
10/10 [==============================] - 12s 1s/step - loss: 2.7878 - accuracy: 0.0950 - val_loss: 2.7139 - val_accuracy: 0.1059
Epoch 5/1000
10/10 [==============================] - 12s 1s/step - loss: 2.7776 - accuracy: 0.1034 - val_loss: 2.7847 - val_accuracy: 0.1059
Epoch 6/1000
10/10 [==============================] - 12s 1s/step - loss: 2.7592 - accuracy: 0.1252 - val_loss: 2.7581 - val_accuracy: 0.1588
Epoch 7/1000
10/10 [==============================] - 12s 1s/step - loss: 2.7416 - accuracy: 0.1261 - val_loss: 2.5202 - val_accuracy: 0.1529
Epoch 

10/10 [==============================] - 12s 1s/step - loss: 1.1990 - accuracy: 0.6328 - val_loss: 0.8329 - val_accuracy: 0.7471
Epoch 59/1000
10/10 [==============================] - 12s 1s/step - loss: 1.1468 - accuracy: 0.6378 - val_loss: 0.4572 - val_accuracy: 0.7706
Epoch 60/1000
10/10 [==============================] - 12s 1s/step - loss: 1.2216 - accuracy: 0.6126 - val_loss: 0.7935 - val_accuracy: 0.7765
Epoch 61/1000
10/10 [==============================] - 12s 1s/step - loss: 1.1377 - accuracy: 0.6521 - val_loss: 0.4072 - val_accuracy: 0.7647
Epoch 62/1000
10/10 [==============================] - 12s 1s/step - loss: 1.1240 - accuracy: 0.6529 - val_loss: 0.5073 - val_accuracy: 0.7882
Epoch 63/1000
10/10 [==============================] - 12s 1s/step - loss: 1.1404 - accuracy: 0.6445 - val_loss: 0.4912 - val_accuracy: 0.7765
Epoch 64/1000
10/10 [==============================] - 12s 1s/step - loss: 1.1061 - accuracy: 0.6563 - val_loss: 0.8356 - val_accuracy: 0.8059
Epoch 65/1000

10/10 [==============================] - 12s 1s/step - loss: 0.6436 - accuracy: 0.7950 - val_loss: 0.0440 - val_accuracy: 0.8941
Epoch 116/1000
10/10 [==============================] - 12s 1s/step - loss: 0.6137 - accuracy: 0.8294 - val_loss: 0.5161 - val_accuracy: 0.8882
Epoch 117/1000
10/10 [==============================] - 12s 1s/step - loss: 0.6544 - accuracy: 0.8042 - val_loss: 0.6285 - val_accuracy: 0.8765
Epoch 118/1000
10/10 [==============================] - 12s 1s/step - loss: 0.6025 - accuracy: 0.8076 - val_loss: 0.5449 - val_accuracy: 0.8941
Epoch 119/1000
10/10 [==============================] - 12s 1s/step - loss: 0.6227 - accuracy: 0.8101 - val_loss: 0.2631 - val_accuracy: 0.9000
Epoch 120/1000
10/10 [==============================] - 12s 1s/step - loss: 0.6141 - accuracy: 0.8160 - val_loss: 0.3034 - val_accuracy: 0.9000
Epoch 121/1000
10/10 [==============================] - 12s 1s/step - loss: 0.6302 - accuracy: 0.8050 - val_loss: 0.2981 - val_accuracy: 0.9000
Epoch 1

In [ ]:
import matplotlib.pyplot as plt

# Plot training & validation accuracy values
plt.plot(hitsory.history['accuracy'])
plt.plot(hitsory.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(hitsory.history['loss'])
plt.plot(hitsory.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
from keras.preprocessing.image import load_img, save_img, img_to_array
import numpy as np

new_datagen = ImageDataGenerator(rescale=1./255)
new_generator = new_datagen.flow_from_directory(
    directory=NEW_DIR,
    target_size=(img_rows, img_cols),
    color_mode='rgb',
    save_to_dir=NEW_DIR,
    class_mode='categorical',
    batch_size=1,
    shuffle=False)

filenames = new_generator.filenames
nb_samples = len(filenames)
print(filenames)
predict = combined_model.predict_generator(new_generator,steps = nb_samples)
print(predict)
for new_flowers in filenames:
    class_name_most_probable = classes[np.argmax(predict[filenames.index(new_flowers)])]
    chance_value_most_probable = np.around(float(np.amax(predict[filenames.index(new_flowers)])), decimals=2)
    print("This flower {} is : {} with chance {}".format(new_flowers.split('\\')[1], class_name_most_probable, chance_value_most_probable))

In [ ]:
from keras.models import load_model
combined_model.save('.\\pretrained_flower_vgg16fc.h5')